# Classification and regression models

In [1]:
import rebound
from spock import FeatureClassifier
feature_model = FeatureClassifier()

sim = rebound.Simulation()
sim.add(m=1.)
sim.add(m=1.e-5, P=1., e=0.03, pomega=2., l=0.5)
sim.add(m=1.e-5, P=1.2, e=0.03, pomega=3., l=3.)
sim.add(m=1.e-5, P=1.5, e=0.03, pomega=1.5, l=2.)
sim.move_to_com()

print(feature_model.predict_stable(sim))

0.06591137


In [2]:
from spock import DeepRegressor
deep_model = DeepRegressor()

median, lower, upper = deep_model.predict_instability_time(sim, samples=10000)
print(int(median))

98722


In [3]:
from spock import class_MLP, CollisionMergerClassifier
class_model = CollisionMergerClassifier()

prob_12, prob_23, prob_13 = class_model.predict_collision_probs(sim)

print('Probability planet 1 collides with 2:', prob_12)
print('Probability planet 2 collides with 3:', prob_23)
print('Probability planet 1 collides with 3:', prob_13)

Probability planet 1 collides with 2: 0.27385116
Probability planet 2 collides with 3: 0.49274087
Probability planet 1 collides with 3: 0.23340799


In [4]:
from spock import reg_MLP, CollisionOrbitalOutcomeRegressor
reg_model = CollisionOrbitalOutcomeRegressor()

a1, a2, e1, e2, inc1, inc2 = reg_model.predict_collision_outcome(sim, collision_inds=[2, 3])

print('Predicted a1:', a1)
print('Predicted e1:', e1, '\n')
print('Predicted a1:', a2)
print('Predicted e1:', e2)

Predicted a1: 0.3362391024283648
Predicted e1: 0.05889093054898315 

Predicted a1: 0.3697486909196322
Predicted e1: 0.029298967242682977


# Giant impact emulator

In [3]:
from spock import GiantImpactPhaseEmulator
from spock import reg_MLP, CollisionOrbitalOutcomeRegressor
from spock import class_MLP, CollisionMergerClassifier
import rebound

In [4]:
sim = rebound.Simulation()
sim.add(m=1.)
sim.add(m=1.e-5, P=1., e=0.03, pomega=2., l=0.5)
sim.add(m=1.e-5, P=1.2, e=0.03, pomega=3., l=3.)
sim.add(m=1.e-5, P=1.5, e=0.03, pomega=1.5, l=2.)
sim.add(m=1.e-5, P=2.2, e=0.03, pomega=0.5, l=5.0)
sim.add(m=1.e-5, P=2.5, e=0.03, pomega=5.0, l=1.5)
sim.move_to_com()

In [5]:
emulator = GiantImpactPhaseEmulator(sim)

In [6]:
%%time
emulator.predict()

CPU times: user 1.74 s, sys: 1.29 s, total: 3.03 s
Wall time: 979 ms


[<rebound.simulation.Simulation object at 0x2a75acb50, N=4, t=999999999.9999993>]

In [6]:
ps = emulator.sims[0].particles
print('t =', emulator.sims[0].t, '\n')
for i in range(1, len(ps)):
    print('Predicted m' + str(i), '=', ps[i].m)
    print('Predicted a' + str(i), '=', ps[i].a)
    print('Predicted e' + str(i), '=', ps[i].e, '\n')

t = 1000000000.0 

Predicted m1 = 2e-05
Predicted a1 = 1.060172215402669
Predicted e1 = 0.03857233228628879 

Predicted m2 = 3.000000000000001e-05
Predicted a2 = 1.7177339319633245
Predicted e2 = 0.03134212650283301 

